In [57]:
# Installation for kafka-python library (run once)
!pip install kafka-python


In [58]:
# Imports and basic configuration
import json
import uuid
import random
import time
from datetime import datetime, timedelta, timezone
from kafka import KafkaProducer
import numpy as np

# Configurable parameters
EVENTS_PER_SECOND = 10  # Rate of events per second
SIMULATION_HOURS = 8   # Duration to run the simulation in hours

In [59]:
# Kafka Endpoint configuration parameters

# KAFKA_BROKER maps to Bootstrap Server address in the Eventstream
# KAFKA_BROKER = ''  # Bootstrap server address

# KAFKA_TOPIC maps to Topic name in the Eventstream
# KAFKA_TOPIC = ''  # Kafka topic for event ingestion

# sas_username to be retained as is.
sas_username = '$ConnectionString'

# sas_password maps to Primary Key Connection String in the Eventstream
# sas_password = ''

In [60]:
producer = None

def init_kafka_producer(
    broker, 
    sasl_username=None, 
    sasl_password=None, 
    sasl_mechanism='PLAIN', 
    security_protocol='SASL_SSL'
):
    global producer
    kafka_config = {
        'bootstrap_servers': [broker],
        'value_serializer': lambda v: json.dumps(v).encode('utf-8'),
        'acks': 'all',
        'retries': 3,
    }
    if sasl_username and sasl_password:
        kafka_config.update({
            'security_protocol': security_protocol,
            'sasl_mechanism': sasl_mechanism,
            'sasl_plain_username': sasl_username,
            'sasl_plain_password': sasl_password
        })
    else:
        kafka_config['security_protocol'] = 'PLAINTEXT'
    
    producer = KafkaProducer(**kafka_config)
    print(f"Kafka producer initialized to broker {broker}")

def send_event_to_kafka(event):
    if producer is None:
        raise Exception("Kafka producer not initialized")
    try:
        future = producer.send(KAFKA_TOPIC, event)
        record_metadata = future.get(timeout=10)
        print(f"Event sent to topic {record_metadata.topic} "
              f"partition {record_metadata.partition} offset {record_metadata.offset}")
        producer.flush()
    except Exception as e:
        print(f"Error sending event to Kafka: {e}")


In [61]:
from enum import Enum
import datetime
import time
import random

# class syntax
class EVENT_TYPE(Enum):
    voltage = 1
    reactive = 2
    current = 3

def generateVoltageEvent():
    event = {}
    event["UNS"] = "yourcompany/nld/helmond/building1/line1/ccSIM"
    event["deviceId"] = "ycCCSIM"
    event["timestamp"] = datetime.datetime.now().isoformat() + "Z"
    event["key"] = EVENT_TYPE.voltage.name   
    event["value"] = str(random.randint(22500, 23500) / 100) # "226.98"
    return event

def generateCurrentEvent():
    event = {}
    event["UNS"] = "yourcompany/nld/helmond/building1/line1/ccSIM"
    event["deviceId"] = "ycCCSIM"
    event["timestamp"] = datetime.datetime.now().isoformat() + "Z"
    event["key"] = EVENT_TYPE.current.name   
    if ( (datetime.datetime.now().minute >= 0) and (datetime.datetime.now().minute < 15) ):
        event["value"] = "0.02"
    if ( (datetime.datetime.now().minute >= 15) and (datetime.datetime.now().minute < 30) ):
        event["value"] = "0.15"
    if ( (datetime.datetime.now().minute >= 30) and (datetime.datetime.now().minute < 45) ):
        event["value"] = "0.03"
    if ( (datetime.datetime.now().minute >= 45) and (datetime.datetime.now().minute < 60) ):
        event["value"] = "0.16"
    return event

def generateReactiveEvent():
    event = {}
    event["UNS"] = "yourcompany/nld/helmond/building1/line1/ccSIM"
    event["deviceId"] = "ycCCSIM"
    event["timestamp"] = datetime.datetime.now().isoformat() + "Z"
    event["key"] = EVENT_TYPE.reactive.name   
    event["value"] = str(random.randint(-650, -550) / 100) # "-0.603"
    return event

In [62]:
def generateEvents(
    kafka_send_func=None, 
    print_events=True
):
    try:
        while True:

            impressionEvent = generateVoltageEvent()    
            if print_events:
                print(json.dumps(impressionEvent))
        
            if kafka_send_func:
                kafka_send_func(impressionEvent)
            time.sleep(0.2)

            impressionEvent = generateCurrentEvent()    
            if print_events:
                print(json.dumps(impressionEvent))
        
            if kafka_send_func:
                kafka_send_func(impressionEvent)
            time.sleep(0.2)

            impressionEvent = generateReactiveEvent()    
            if print_events:
                print(json.dumps(impressionEvent))
        
            if kafka_send_func:
                kafka_send_func(impressionEvent)
            time.sleep(0.2)

            time.sleep(1)
    except KeyboardInterrupt:
        producer_events.close()

In [63]:
# Initialize Kafka Producer with SASL credentials
init_kafka_producer(KAFKA_BROKER, sas_username, sas_password)

# Run simulation with event sending to Kafka enabled, printing events to notebook
#run_event_simulation(
#    EVENTS_PER_SECOND,
#    SIMULATION_HOURS,
#    kafka_send_func=send_event_to_kafka,
#    print_events=True
#)

print(datetime.datetime.now())

generateEvents( 
    kafka_send_func=send_event_to_kafka,
    print_events=True
)

print(datetime.datetime.now())

<BrokerConnection client_id=kafka-python-producer-9, node_id=bootstrap-0 host=esehamhh6vkjpj4antusqh.servicebus.windows.net:9093 <checking_api_versions_recv> [IPv4 ('20.86.89.26', 9093)]>: socket disconnected
<BrokerConnection client_id=kafka-python-producer-9, node_id=bootstrap-0 host=esehamhh6vkjpj4antusqh.servicebus.windows.net:9093 <checking_api_versions_recv> [IPv4 ('20.86.89.26', 9093)]>: Closing connection. KafkaConnectionError: socket disconnected
No node available during check_version; sleeping 0.01 secs


Kafka producer initialized to broker esehamhh6vkjpj4antusqh.servicebus.windows.net:9093
2025-10-03 12:17:37.919165
{"UNS": "yourcompany/nld/helmond/building1/line1/ccSIM", "deviceId": "ycCCSIM", "timestamp": "2025-10-03T12:17:37.919228Z", "key": "voltage", "value": "228.59"}
Event sent to topic es_1c25fcef-67a0-41fc-a0bb-97370625f701 partition 1 offset 102
{"UNS": "yourcompany/nld/helmond/building1/line1/ccSIM", "deviceId": "ycCCSIM", "timestamp": "2025-10-03T12:17:38.235720Z", "key": "current", "value": "0.15"}
Event sent to topic es_1c25fcef-67a0-41fc-a0bb-97370625f701 partition 1 offset 103
{"UNS": "yourcompany/nld/helmond/building1/line1/ccSIM", "deviceId": "ycCCSIM", "timestamp": "2025-10-03T12:17:38.444746Z", "key": "reactive", "value": "-6.3"}
Event sent to topic es_1c25fcef-67a0-41fc-a0bb-97370625f701 partition 2 offset 99
{"UNS": "yourcompany/nld/helmond/building1/line1/ccSIM", "deviceId": "ycCCSIM", "timestamp": "2025-10-03T12:17:39.657977Z", "key": "voltage", "value": "225.3